In [58]:
import os
import collections
import re
import pprint

# Importing data

In [2]:
FILES_1 = [file for file in os.listdir("logs_uniform") if file[-6:] == "_2.log"]

In [3]:
FILES_2 = [file for file in os.listdir("logs_uniform2") if file[-6:] == "_2.log"]

In [26]:
len(FILES_1) + len(FILES_2)

26

In [27]:
records_list = list()

for file in FILES_1:
    f = open(f"logs_uniform/{file}", "r")
    for line in f:
        records_list.append(line.strip().split(";"))

for file in FILES_2:
    f = open(f"logs_uniform2/{file}", "r")
    for line in f:
        records_list.append(line.strip().split(";"))

In [29]:
len(records_list)

240001

In [31]:
records_list[3]

['4',
 'SUCCESS',
 '0',
 '8.0',
 '8.0',
 '0',
 '1',
 'EditDeletion-EmptyPatch',
 'None',
 'None',
 "{'test_6(java_testcases.junit.LCS_LENGTH_TEST): null', 'test_5(java_testcases.junit.LCS_LENGTH_TEST): null', 'test_8(java_testcases.junit.LCS_LENGTH_TEST): null', 'test_0(java_testcases.junit.LCS_LENGTH_TEST): null', 'test_7(java_testcases.junit.LCS_LENGTH_TEST): null', 'test_4(java_testcases.junit.LCS_LENGTH_TEST): null', 'test_1(java_testcases.junit.LCS_LENGTH_TEST): null', 'test_3(java_testcases.junit.LCS_LENGTH_TEST): null'}"]

In [32]:
for line in records_list:
    
    if line[10] != "None":
        line[10] = line[10].split(", ")
        line[10][0] = line[10][0][1:]
        line[10][-1] = line[10][-1][:-2]
        line[10] = set(line[10])

In [36]:
records_list[1]

['2',
 'PARSE_ERROR',
 '1',
 'None',
 '8.0',
 '1',
 '0',
 'StmtReplacement',
 './for[2]/for[1]/if[1]/if[1]/decl_stmt[1]',
 './for[2]/for[1]/if[1]/if[1]',
 'None']

# Aggregating data: operator x target node

0 iter, 1 run status, 2 return code, 3 current fitness, 4 previous fitness, 5 current patch, 6 previous patch, 7 operator, 8 target, 9 ingredient, 10 failures info

aggr_dict[operator] = {node_type: [better fitness, worse fitness, same fitness]}

In [109]:
aggr_dict = collections.defaultdict(dict)

iter_per_operator = collections.defaultdict(int)

for line in records_list:
    operator = line[7]
    
    iter_per_operator[operator] += 1
    
    if not "EditDeletion" in operator: # and operator != "ComparisonOperator": 
        target = line[8].split("/")[-1][:-3]
        fitness = line[3]
        previous_fitness = line[4]
        tests_info = line[10]
        
        if not target in aggr_dict[operator].keys():
            aggr_dict[operator][target] = [0, 0, 0]
        
        if fitness < previous_fitness:
            aggr_dict[operator][target][0] += 1
        elif fitness > previous_fitness:
            aggr_dict[operator][target][1] += 1
        else:
            aggr_dict[operator][target][2] += 1

In [110]:
for key, value in iter_per_operator.items():
    print(key, value, make_percent(value / len(records_list)))

StmtDeletion 7513 3.13%
StmtReplacement 85436 35.6%
EditDeletion-EmptyPatch 24020 10.01%
StmtInsertion 85860 35.77%
ComparisonOperator 6747 2.81%
EditDeletion 30425 12.68%


In [111]:
pp = pprint.PrettyPrinter(indent=1)
pp.pprint(aggr_dict)

defaultdict(<class 'dict'>,
            {'ComparisonOperator': {'operator_comp': [824, 2574, 3349]},
             'StmtDeletion': {'continue': [0, 0, 7],
                              'decl_stmt': [0, 1691, 80],
                              'expr_stmt': [51, 973, 821],
                              'for': [57, 299, 293],
                              'if': [130, 963, 231],
                              'return': [23, 1558, 183],
                              'while': [0, 113, 40]},
             'StmtInsertion': {'continue': [0, 76, 66],
                               'decl_stmt': [15, 18264, 2401],
                               'expr_stmt': [135, 15592, 7815],
                               'for': [41, 7066, 1645],
                               'if': [139, 10596, 3395],
                               'return': [199, 11909, 4622],
                               'while': [2, 1527, 355]},
             'StmtReplacement': {'continue': [0, 71, 69],
                                 'decl_s

In [112]:
def make_percent(num):
    num = num * 100
    num = round(num, 2)
    
    return str(num) + "%"

In [113]:
percentage_aggr_dict = collections.defaultdict(dict)

for operator in aggr_dict.keys():
    for target in aggr_dict[operator].keys():
        stats = aggr_dict[operator][target]
        
        percentage_aggr_dict[operator][target] = [
            make_percent(stats[0] / sum(stats)), 
            make_percent(stats[1] / sum(stats)), 
           make_percent( stats[2] / sum(stats))
        ]

In [114]:
pp = pprint.PrettyPrinter(indent=1)
pp.pprint(percentage_aggr_dict)

defaultdict(<class 'dict'>,
            {'ComparisonOperator': {'operator_comp': ['12.21%',
                                                      '38.15%',
                                                      '49.64%']},
             'StmtDeletion': {'continue': ['0.0%', '0.0%', '100.0%'],
                              'decl_stmt': ['0.0%', '95.48%', '4.52%'],
                              'expr_stmt': ['2.76%', '52.74%', '44.5%'],
                              'for': ['8.78%', '46.07%', '45.15%'],
                              'if': ['9.82%', '72.73%', '17.45%'],
                              'return': ['1.3%', '88.32%', '10.37%'],
                              'while': ['0.0%', '73.86%', '26.14%']},
             'StmtInsertion': {'continue': ['0.0%', '53.52%', '46.48%'],
                               'decl_stmt': ['0.07%', '88.32%', '11.61%'],
                               'expr_stmt': ['0.57%', '66.23%', '33.2%'],
                               'for': ['0.47%', '80.74%', '18.8%']

In [115]:
improved_fitness_sum = 0
improved_fitness_per_operator = collections.defaultdict(int)

for operator in aggr_dict.keys():
    for target in aggr_dict[operator].keys():
        improved_fitness_sum += aggr_dict[operator][target][0]
        improved_fitness_per_operator[operator] += aggr_dict[operator][target][0]

In [116]:
improved_fitness_sum

2735

In [117]:
for key, value in improved_fitness_per_operator.items():
    print(key, value, make_percent(value / improved_fitness_sum))

StmtDeletion 261 9.54%
StmtReplacement 1119 40.91%
StmtInsertion 531 19.41%
ComparisonOperator 824 30.13%


In [118]:
for operator in aggr_dict.keys():
    for target in aggr_dict[operator].keys():
        print(operator, target, "-", make_percent(aggr_dict[operator][target][0]/improved_fitness_sum))

StmtDeletion decl_stmt - 0.0%
StmtDeletion expr_stmt - 1.86%
StmtDeletion if - 4.75%
StmtDeletion for - 2.08%
StmtDeletion return - 0.84%
StmtDeletion while - 0.0%
StmtDeletion continue - 0.0%
StmtReplacement decl_stmt - 0.0%
StmtReplacement for - 4.83%
StmtReplacement return - 11.7%
StmtReplacement expr_stmt - 14.15%
StmtReplacement if - 10.2%
StmtReplacement while - 0.04%
StmtReplacement continue - 0.0%
StmtInsertion expr_stmt - 4.94%
StmtInsertion if - 5.08%
StmtInsertion decl_stmt - 0.55%
StmtInsertion return - 7.28%
StmtInsertion for - 1.5%
StmtInsertion while - 0.07%
StmtInsertion continue - 0.0%
ComparisonOperator operator_comp - 30.13%


# Aggregating data: operator x target node x ingredient node

0 iter, 1 run status, 2 return code, 3 current fitness, 4 previous fitness, 5 current patch, 6 previous patch, 7 operator, 8 target, 9 ingredient, 10 failures info